In [176]:
import pandas as pd
import numpy as np
import json
pd.options.mode.chained_assignment = None

In [177]:
INPUT_FILENAME = 'api_submission.csv'
EXPORT_FILENAME = 'api_submission_processed.csv'

df = pd.read_csv(INPUT_FILENAME)
df

,submission_code,date,is_passed,account_id,problem_id,max_score,passed_ratio,score,language,topic_id,submission_id
0,"print(""Hello World!"")\n",2022-10-27 07:16:02.229668,1,5,26,0,0.0,0,python,NaN,160
1,NaN,2022-10-27 07:16:16.675488,0,5,26,0,0.0,0,python,NaN,161
2,"print(""Hello World!"")\n",2022-10-27 07:16:19.502776,1,5,26,0,0.0,0,python,NaN,162
3,"print(""Hello World!"")\n",2022-10-27 07:17:19.152737,1,5,26,0,0.0,0,python,NaN,163
4,"print(""Hello World"")\n",2022-11-13 11:56:38.030682,0,4,26,0,0.0,0,python,NaN,182
...,...,...,...,...,...,...,...,...,...,...,...
2185,#include <stdio.h>\r\n\r\nint main() {\r\n ...,2024-01-14 13:27:12.822191,0,14,466254a3a6ac4bc8b7ee9e0925d2ba86,10,0.2,2,c,538b9db3e2a04c94974c54526115f844,243658b154a34bb9bb87268400bf25a5
2186,#include <stdio.h>\r\n\r\nint main() {\r\n ...,2024-01-15 20:43:13.728600,0,4,466254a3a6ac4bc8b7ee9e0925d2ba86,10,0.0,0,c,538b9db3e2a04c94974c54526115f844,de692bd7b9374bc9a2346fae8ec92a9e
2187,#include <stdio.h>\r\n\r\nint main() {\r\n ...,2024-01-15 20:43:26.187614,0,4,466254a3a6ac4bc8b7ee9e0925d2ba86,10,0.0,0,c,538b9db3e2a04c94974c54526115f844,cd39f2e6d58c4fb396507752804e3a1c
2188,#include <stdio.h>\r\n\r\nint main() {\r\n ...,2024-01-15 20:44:00.234384,0,4,466254a3a6ac4bc8b7ee9e0925d2ba86,10,0.2,2,c,538b9db3e2a04c94974c54526115f844,88d691f7b6714c61a1c127fe28eee1e8


# Data Preprocessing

In [178]:
account_df = df[['account_id', 'problem_id','is_passed','language','date','submission_code','passed_ratio']]

# Change submission_code to string
account_df['submission_code'] = account_df['submission_code'].astype(str)
account_df

,account_id,problem_id,is_passed,language,date,submission_code,passed_ratio
0,5,26,1,python,2022-10-27 07:16:02.229668,"print(""Hello World!"")\n",0.0
1,5,26,0,python,2022-10-27 07:16:16.675488,nan,0.0
2,5,26,1,python,2022-10-27 07:16:19.502776,"print(""Hello World!"")\n",0.0
3,5,26,1,python,2022-10-27 07:17:19.152737,"print(""Hello World!"")\n",0.0
4,4,26,0,python,2022-11-13 11:56:38.030682,"print(""Hello World"")\n",0.0
...,...,...,...,...,...,...,...
2185,14,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-14 13:27:12.822191,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.2
2186,4,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-15 20:43:13.728600,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.0
2187,4,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-15 20:43:26.187614,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.0
2188,4,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-15 20:44:00.234384,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.2


In [179]:
# Sort by account_id, problem_id and date
account_df.sort_values(['account_id','problem_id','date'])

# Calculate the difference time between the current submission and the previous submission
account_df['date'] = pd.to_datetime(account_df['date'])
account_df['diff_time'] = account_df.groupby(['account_id','problem_id'])['date'].diff()
account_df['diff_time'] = account_df['diff_time'].dt.total_seconds()

# Set all diff_time which is NaN to 0
account_df['diff_time'].fillna(0, inplace=True)

# Set all diff_time which has more than 3 hours to 3
account_df.loc[account_df['diff_time'] > 10800, 'diff_time'] = 10801
account_df

,account_id,problem_id,is_passed,language,date,submission_code,passed_ratio,diff_time
0,5,26,1,python,2022-10-27 07:16:02.229668,"print(""Hello World!"")\n",0.0,0.000000
1,5,26,0,python,2022-10-27 07:16:16.675488,nan,0.0,14.445820
2,5,26,1,python,2022-10-27 07:16:19.502776,"print(""Hello World!"")\n",0.0,2.827288
3,5,26,1,python,2022-10-27 07:17:19.152737,"print(""Hello World!"")\n",0.0,59.649961
4,4,26,0,python,2022-11-13 11:56:38.030682,"print(""Hello World"")\n",0.0,0.000000
...,...,...,...,...,...,...,...,...
2185,14,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-14 13:27:12.822191,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.2,8.971618
2186,4,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-15 20:43:13.728600,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.0,0.000000
2187,4,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-15 20:43:26.187614,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.0,12.459014
2188,4,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-15 20:44:00.234384,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.2,34.046770


In [180]:
# Loop each row
current_problem_id = None
current_account_id = None
already_passed = False
previous_diff_time = -1
for index, row in account_df.iterrows():
    if (not current_problem_id and not current_account_id) or current_problem_id != row['problem_id'] and current_account_id != row['account_id']:
        current_problem_id = row['problem_id']
        current_account_id = row['account_id']
        already_passed = False
    elif already_passed:
        account_df.drop(index, inplace=True)
    elif row['is_passed'] == 1:
        already_passed = True
        if row['diff_time'] > 10800:
            row['diff_time'] = previous_diff_time
account_df

,account_id,problem_id,is_passed,language,date,submission_code,passed_ratio,diff_time
0,5,26,1,python,2022-10-27 07:16:02.229668,"print(""Hello World!"")\n",0.0,0.000000
1,5,26,0,python,2022-10-27 07:16:16.675488,nan,0.0,14.445820
2,5,26,1,python,2022-10-27 07:16:19.502776,"print(""Hello World!"")\n",0.0,2.827288
7,4,33,1,python,2022-11-14 07:35:00.337364,"text = input(""Input text: "")\ncol = int(input(...",0.0,0.000000
8,4,31,1,python,2022-11-14 07:35:35.289750,"password = input(""Enter a password: "")\n\nleng...",0.0,0.000000
...,...,...,...,...,...,...,...,...
2185,14,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-14 13:27:12.822191,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.2,8.971618
2186,4,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-15 20:43:13.728600,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.0,0.000000
2187,4,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-15 20:43:26.187614,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.0,12.459014
2188,4,466254a3a6ac4bc8b7ee9e0925d2ba86,0,c,2024-01-15 20:44:00.234384,#include <stdio.h>\r\n\r\nint main() {\r\n ...,0.2,34.046770


In [181]:
# Group problem_id and account_id
grouped_account_df = account_df.groupby(['account_id','problem_id']).agg({'is_passed':'max','diff_time':'sum','passed_ratio':'count'})

# Change passed_ratio to submissions_count
grouped_account_df.rename(columns={'passed_ratio':'submissions_count'}, inplace=True)
grouped_account_df

is_passed     diff_time  submissions_count
account_id problem_id                                            
4          101                 1      0.000000                  1
           102                 0   2274.243600                 10
           103                 1      0.000000                  1
           111                 0      0.000000                  1
           113                 0  10801.000000                  2
...                          ...           ...                ...
34         90                  1  14344.738131                  5
           91                  0  12335.630535                  8
35         39                  1    591.711660                  8
37         39                  1    379.964636                 14
           89                  1   1544.946604                 10

[429 rows x 3 columns]

In [182]:
# Pivot table: Column -> account_id_is_passed. account_id_diff_time, Row -> problem_id
pv = grouped_account_df.pivot_table(index=['problem_id'], columns=['account_id'], values=['is_passed','diff_time','submissions_count'], aggfunc='mean')

# Replace all NaN to 0
pv.fillna(0, inplace=True)

# Merge column names (account_id_is_passed, account_id_diff_time, account_id_submissions_count) to one column name
pv.columns = pv.columns.map('{0[0]}_{0[1]}'.format)
pv

,diff_time_4,diff_time_5,diff_time_10,diff_time_11,diff_time_12,diff_time_13,diff_time_14,diff_time_15,diff_time_16,diff_time_17,...,submissions_count_24,submissions_count_26,submissions_count_29,submissions_count_30,submissions_count_31,submissions_count_32,submissions_count_33,submissions_count_34,submissions_count_35,submissions_count_37
problem_id,,,,,,,,,,,,,,,,,,,,,
100,0.0000,0.0,0.0,0.0,0.0,150.091243,0.000000,0.0,0.0,0.0,...,8.0,0.0,0.0,3.0,0.0,11.0,0.0,0.0,0.0,0.0
101,0.0000,0.0,0.0,0.0,0.0,107.800951,1144.985866,0.0,0.0,0.0,...,1.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
102,2274.2436,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,0.0000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,0.0000,0.0,0.0,0.0,0.0,0.000000,279.498055,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0000,0.0,0.0,0.0,0.0,0.000000,160.537668,0.0,0.0,0.0,...,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0000,0.0,0.0,0.0,0.0,411.641592,0.000000,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0000,0.0,0.0,0.0,0.0,63.105462,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Part II: Preprocessing on Problem attributes

In [183]:
problem_df = pd.read_csv('api_problem.csv')
problem_df

,problem_id,language,title,description,solution,time_limit,is_private,is_active,submission_regex,creator_id,created_date,updated_date,sharing,allowed_languages
0,26,py,Hello World,"[{""id"":""1"",""type"":""p"",""children"":[{""text"":""""}]}]","print(""Hello World!"")",1.5,1,1,.*,4,2024-01-04 18:13:22.044451,2024-01-12 11:08:49.196553,0,"python,c,cpp"
1,30,py,Palindrome,"[{""id"":""1"",""type"":""p"",""children"":[{""text"":""Pal...","text = input(""Enter a word: "")\n\nfor i in ran...",1.5,0,1,.*,4,2024-01-04 18:13:22.044451,2024-01-12 05:12:09.403031,0,"python,c,cpp"
2,31,py,Password Validation,"[{""id"":""1"",""type"":""p"",""children"":[{""text"":""ในก...","password = input(""Enter a password: "")\n\nleng...",1.5,0,1,.*,4,2024-01-04 18:13:22.044451,2024-01-04 18:13:22.062519,0,"python,c,cpp"
3,32,py,Text Wrap,เขียนโปรแกรมที่รับข้อความเข้ามา และจำนวนเต็ม 1...,"s = input(""Input text: "")\nn = int(input(""N: ""...",1.5,0,1,.*,4,2024-01-04 18:13:22.044451,2024-01-04 18:13:22.062519,0,"python,c,cpp"
4,33,py,Text Transpose,"[{""children"":[{""text"":""เขียนโปรแกรมที่รับข้อคว...","text = input(""Input text: "")\ncol = int(input(...",1.5,0,1,.*,4,2024-01-04 18:13:22.044451,2024-01-04 18:13:22.062519,0,"python,c,cpp"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,941c471296114a13872a09b6ae18f98c,py,Plus (C),"[{""id"":""1"",""type"":""p"",""children"":[{""text"":""เขี...",\na = int(input())\nb = int(input())\n\nprint(...,1.5,0,1,.*,4,2024-01-14 08:35:55.776094,2024-01-14 08:38:52.767052,0,c
141,466254a3a6ac4bc8b7ee9e0925d2ba86,py,Math Operation (C),"[{""type"":""p"",""align"":""start"",""children"":[{""tex...",\na = int(input())\nb = int(input())\n\nprint(...,1.5,0,1,.*,4,2024-01-14 08:36:18.437943,2024-01-15 20:45:08.770987,0,c
142,88d30780f27e4527bce17178240ecf4a,py,Even Odd (C),"[{""type"":""p"",""align"":""start"",""children"":[{""tex...",\nn = int(input())\n\nif n % 2 == 0:\n prin...,1.5,0,1,.*,4,2024-01-14 08:36:47.603068,2024-01-14 08:40:43.178403,0,c
143,a8c0f56288844805b602fbe58476fc07,py,Days In Month (C),"[{""id"":""1"",""type"":""p"",""children"":[{""text"":""เขี...",\nm = int(input())\ny = int(input())\ny-=543\n...,1.5,0,1,.*,4,2024-01-14 08:37:16.107468,2024-01-14 08:47:38.747713,0,c


In [184]:
pf_ratio_df = df[['problem_id','is_passed']]
pf_ratio_df = pf_ratio_df.groupby(['problem_id']).agg({'is_passed':'mean'})
pf_ratio_df.rename(columns={'is_passed':'passed_failed_ratio'}, inplace=True)
pf_ratio_df

,passed_failed_ratio
problem_id,
100,0.250000
101,0.428571
102,0.148148
103,0.454545
104,0.700000
...,...
96,0.400000
97,0.210526
98,0.470588


In [185]:
time_limit_df = problem_df[['problem_id','time_limit']]
time_limit_df = time_limit_df.groupby(['problem_id']).agg({'time_limit':'mean'})
time_limit_df

,time_limit
problem_id,
05a7775c49a245e4a3e943bafc0ce0f4,1.0
100,1.5
101,1.5
102,5.0
103,1.5
...,...
98,1.5
99,1.5
a8c0f56288844805b602fbe58476fc07,1.5


In [186]:
allowed_lang_df = problem_df[['problem_id','allowed_languages']]
# Pivot earh value on allowed language to column
allowed_lang_df = allowed_lang_df.join(allowed_lang_df.allowed_languages.str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('allowed_language'))
allowed_lang_df.drop(columns=['allowed_languages'], inplace=True)

allowed_lang_df = allowed_lang_df.pivot_table(index=['problem_id'], columns=['allowed_language'], aggfunc='size', fill_value=0)
# Merge column name
allowed_lang_df

allowed_language,c,cpp,python
problem_id,,,
05a7775c49a245e4a3e943bafc0ce0f4,1,1,1
100,1,1,1
101,1,1,1
102,1,1,1
103,1,1,1
...,...,...,...
98,1,1,1
99,1,1,1
a8c0f56288844805b602fbe58476fc07,1,0,0


In [187]:
difficulty = json.load(open('difficulty.json'))

# Add difficulty column based on thier problem_id
def applyDifficulty(row):
    return difficulty[str(row.name)]

def applyPassedFailedRatio(row):
    return pf_ratio_df.loc[row.name]['passed_failed_ratio']



pv['passed_failed_ratio'] = pv.apply(applyPassedFailedRatio, axis=1)
pv['difficulty'] = pv.apply(applyDifficulty, axis=1)

# Join pv with allowed_lang_df by problem_id
pv = pv.join(allowed_lang_df)
pv = pv.join(time_limit_df)
pv

,diff_time_4,diff_time_5,diff_time_10,diff_time_11,diff_time_12,diff_time_13,diff_time_14,diff_time_15,diff_time_16,diff_time_17,...,submissions_count_33,submissions_count_34,submissions_count_35,submissions_count_37,passed_failed_ratio,difficulty,c,cpp,python,time_limit
problem_id,,,,,,,,,,,,,,,,,,,,,
100,0.0000,0.0,0.0,0.0,0.0,150.091243,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.250000,1,1,1,1,1.5
101,0.0000,0.0,0.0,0.0,0.0,107.800951,1144.985866,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.428571,1,1,1,1,1.5
102,2274.2436,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.148148,2,1,1,1,5.0
103,0.0000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.454545,1,1,1,1,1.5
104,0.0000,0.0,0.0,0.0,0.0,0.000000,279.498055,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.700000,1,1,1,1,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0000,0.0,0.0,0.0,0.0,0.000000,160.537668,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.400000,1,1,1,1,1.5
97,0.0000,0.0,0.0,0.0,0.0,411.641592,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.210526,1,1,1,1,1.5
98,0.0000,0.0,0.0,0.0,0.0,63.105462,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.470588,1,1,1,1,1.5


In [188]:
# Write csv
pv.to_csv(EXPORT_FILENAME)
print(f"Wrote file: {EXPORT_FILENAME}")